In [4]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from transformers import pipeline
import requests
from io import BytesIO
import os


2025-04-30 17:10:33.432359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746022233.517886   12803 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746022233.541802   12803 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746022233.720206   12803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746022233.720254   12803 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746022233.720255   12803 computation_placer.cc:177] computation placer alr

In [5]:
# Skin disease class names
class_names = [
    'Basal Cell Carcinoma', 'Darier_s Disease', 'Epidermolysis Bullosa Pruriginosa',
    'Hailey-Hailey Disease', 'Herpes Simplex', 'Impetigo', 'Larva Migrans',
    'Leprosy Borderline', 'Leprosy Lepromatous', 'Leprosy Tuberculoid',
    'Lichen Planus', 'Lupus Erythematosus Chronicus Discoides', 'Melanoma',
    'Molluscum Contagiosum', 'Mycosis Fungoides', 'Neurofibromatosis',
    'Papilomatosis Confluentes And Reticulate', 'Pediculosis Capitis',
    'Pityriasis Rosea', 'Porokeratosis Actinic', 'Psoriasis', 'Tinea Corporis',
    'Tinea Nigra', 'Tungiasis', 'actinic keratosis', 'dermatofibroma', 'nevus',
    'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma',
    'vascular lesion'
]

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [6]:
def load_classification_model(model_path):
    try:
        model = torch.jit.load(model_path)
        model.eval()
        print(f"✅ Classification model loaded from {model_path}")
        return model
    except Exception as e:
        print(f"❌ Error loading classification model: {e}")
        return None

# Example: Replace with your actual model path
model_path = "model_mobile.pt"
model = load_classification_model(model_path)


✅ Classification model loaded from model_mobile.pt


In [7]:
def classify_image(model, image, class_names):
    if isinstance(image, str):
        if image.startswith('http'):
            response = requests.get(image)
            image = Image.open(BytesIO(response.content)).convert("RGB")
        else:
            image = Image.open(image).convert("RGB")
    
    input_tensor = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
    
    probs = F.softmax(output, dim=-1)
    predicted_class_idx = probs.argmax(-1).item()
    confidence = probs[0, predicted_class_idx].item()

    top_probs, top_indices = torch.topk(probs, 3, dim=1)
    top_predictions = [
        {"class": class_names[idx], "confidence": prob.item() * 100}
        for prob, idx in zip(top_probs[0], top_indices[0])
    ]

    predicted_class_name = class_names[predicted_class_idx]

    return {
        "predicted_class": predicted_class_name,
        "confidence": confidence * 100,
        "top_predictions": top_predictions
    }


In [8]:
def load_llm(llm_model="TinyLlama/TinyLlama-1.1B-Chat-v1.0"):
    try:
        llm = pipeline("text-generation", model=llm_model, device="cpu")  # Explicitly set to CPU
        print(f"✅ LLM loaded on CPU: {llm_model}")
        return llm
    except Exception as e:
        print(f"❌ Error loading LLM: {e}")
        return None

llm = load_llm()


Device set to use cpu


✅ LLM loaded on CPU: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [9]:
def generate_llm_prompt(classification_result, user_question=None):
    predicted_class = classification_result["predicted_class"]
    confidence = classification_result["confidence"]

    system_prompt = f"""As a dermatology assistant, I've analyzed an image and detected {predicted_class} with {confidence:.2f}% confidence. 
My secondary predictions are: {', '.join([f"{p['class']} ({p['confidence']:.2f}%)" for p in classification_result['top_predictions'][1:]])}. 
Based on this analysis, provide helpful, factual medical information about {predicted_class}. 
IMPORTANT: Always advise the patient to consult with a healthcare professional for proper diagnosis and treatment."""

    if user_question:
        return f"{system_prompt}\n\nPatient question: {user_question}"
    else:
        return f"{system_prompt}\n\nProvide basic information about this condition, its common treatments, and necessary precautions."


In [10]:
def generate_response(llm, classification_result, user_question=None):
    if llm is None:
        return "LLM is not available. Please check model initialization."
    
    prompt = generate_llm_prompt(classification_result, user_question)
    messages = [{"role": "user", "content": prompt}]
    
    try:
        response = llm(messages, max_length=500, temperature=0.7)
        return response[0]['generated_text']
    except Exception as e:
        return f"Error generating response: {e}"


In [11]:
def process_image_and_question(model, llm, image_path, user_question=None):
    classification_result = classify_image(model, image_path, class_names)
    llm_response = generate_response(llm, classification_result, user_question)
    return classification_result, llm_response


In [12]:
image_path = "/home/rabieash/projects/GP/psoriasis-treatment-in-houston.jpg"
user_question = "What are the treatments for this condition and how serious is it?"

if os.path.exists(image_path):
    classification, response = process_image_and_question(model, llm, image_path, user_question)

    print("\n----- CLASSIFICATION RESULTS -----")
    print(f"Detected: {classification['predicted_class']} (Confidence: {classification['confidence']:.2f}%)")

    print("\n----- AI ASSISTANT RESPONSE -----")
    print(response)
else:
    print(f"Image not found at {image_path}")



----- CLASSIFICATION RESULTS -----
Detected: Psoriasis (Confidence: 91.63%)

----- AI ASSISTANT RESPONSE -----
[{'role': 'user', 'content': "As a dermatology assistant, I've analyzed an image and detected Psoriasis with 91.63% confidence. \nMy secondary predictions are: Lichen Planus (4.13%), Hailey-Hailey Disease (1.18%). \nBased on this analysis, provide helpful, factual medical information about Psoriasis. \nIMPORTANT: Always advise the patient to consult with a healthcare professional for proper diagnosis and treatment.\n\nPatient question: What are the treatments for this condition and how serious is it?"}, {'role': 'assistant', 'content': "As a dermatology assistant, you have analyzed an image of Psoriasis and detected the following secondary predictions: Lichen Planus (4.13%), Hailey-Hailey Disease (1.18%).\n\nPsoriasis is a chronic, inflammatory skin condition that affects the skin's surface, causing red, itchy, and scaly patches. It is a common skin disorder that affects mill

In [15]:
image_path = "/home/rabieash/projects/GP/Pityriasis rosea.jpg"
user_question = "What are the treatments for this condition and how serious is it"

if os.path.exists(image_path):
    classification, response = process_image_and_question(model, llm, image_path, user_question)

    print("\n----- CLASSIFICATION RESULTS -----")
    print(f"Detected: {classification['predicted_class']} (Confidence: {classification['confidence']:.2f}%)")

    print("\n----- AI ASSISTANT RESPONSE -----")
    print(response)
else:
    print(f"Image not found at {image_path}")



----- CLASSIFICATION RESULTS -----
Detected: Pityriasis Rosea (Confidence: 100.00%)

----- AI ASSISTANT RESPONSE -----
[{'role': 'user', 'content': "As a dermatology assistant, I've analyzed an image and detected Pityriasis Rosea with 100.00% confidence. \nMy secondary predictions are: Tinea Corporis (0.00%), Psoriasis (0.00%). \nBased on this analysis, provide helpful, factual medical information about Pityriasis Rosea. \nIMPORTANT: Always advise the patient to consult with a healthcare professional for proper diagnosis and treatment.\n\nPatient question: What are the treatments for this condition and how serious is it"}, {'role': 'assistant', 'content': "Pityriasis Rosea is a common skin condition characterized by a red, itchy, and scaly rash that typically appears on the trunk, arms, and legs. It is caused by a fungus called Pityrosporum ovale, which can affect people of all ages and races. Treatment options for Pityriasis Rosea include topical antifungal medications, such as ketoc

In [9]:
def ask_patient_info():
    age = input("Enter your age: ")
    sex = input("Enter your sex (Male/Female/Other): ")
    return {
        "age": age.strip(),
        "sex": sex.strip().capitalize()
    }

patient_info = ask_patient_info()


In [10]:
def generate_llm_prompt(classification_result, user_question=None, patient_info=None):
    predicted_class = classification_result["predicted_class"]
    confidence = classification_result["confidence"]

    # Add patient context if provided
    patient_context = ""
    if patient_info:
        patient_context = (
            "Patient details:\n"
            f"- Age: {patient_info.get('age', 'N/A')}\n"
            f"- Sex: {patient_info.get('sex', 'N/A')}\n"
            "\nTailor your response to the patient's age and sex when discussing risks, "
            "treatment options, or common symptoms."
        )

    # Base prompt
    system_prompt = f"""As a dermatology assistant, I've analyzed an image and detected {predicted_class} with {confidence:.2f}% confidence.
My secondary predictions are: {', '.join([f"{p['class']} ({p['confidence']:.2f}%)" for p in classification_result['top_predictions'][1:]])}.
{patient_context}

Based on this analysis, provide helpful, factual medical information about {predicted_class}.
IMPORTANT: Always advise the patient to consult with a healthcare professional for proper diagnosis and treatment."""

    if user_question:
        return f"{system_prompt}\n\nPatient question: {user_question}"
    else:
        return f"{system_prompt}\n\nProvide basic information about this condition, its common treatments, and necessary precautions."


In [11]:
def generate_response(llm, classification_result, user_question=None, patient_info=None):
    if llm is None:
        return "LLM is not available. Please check model initialization."

    # Only add user question if provided
    prompt = generate_llm_prompt(classification_result, user_question, patient_info)
    messages = [{"role": "user", "content": prompt}]
    
    try:
        # Set truncation=True to avoid truncation warnings
        response = llm(messages, max_length=500, temperature=0.7, truncation=True)
        return response[0]['generated_text']
    except Exception as e:
        return f"Error generating response: {e}"



In [12]:
def process_image_and_question(model, llm, image_path, user_question=None, patient_info=None):
    classification_result = classify_image(model, image_path, class_names)
    llm_response = generate_response(llm, classification_result, user_question, patient_info)
    return classification_result, llm_response

In [14]:
image_path = "//home/rabieash/projects/GP/Pityriasis rosea.jpg"
user_question = "What are the treatments for this condition and how serious is it"

if os.path.exists(image_path):
    classification, response = process_image_and_question(model, llm, image_path, user_question, patient_info)

    print("\n----- CLASSIFICATION RESULTS -----")
    print(f"Detected: {classification['predicted_class']} (Confidence: {classification['confidence']:.2f}%)")

    print("\n----- AI ASSISTANT RESPONSE -----")
    print(response)
else:
    print(f"Image not found at {image_path}")


/home/rabieash/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



----- CLASSIFICATION RESULTS -----
Detected: Pityriasis Rosea (Confidence: 100.00%)

----- AI ASSISTANT RESPONSE -----
[{'role': 'user', 'content': "As a dermatology assistant, I've analyzed an image and detected Pityriasis Rosea with 100.00% confidence.\nMy secondary predictions are: Tinea Corporis (0.00%), Psoriasis (0.00%).\nPatient details:\n- Age: \n- Sex: 22\n\nTailor your response to the patient's age and sex when discussing risks, treatment options, or common symptoms.\n\nBased on this analysis, provide helpful, factual medical information about Pityriasis Rosea.\nIMPORTANT: Always advise the patient to consult with a healthcare professional for proper diagnosis and treatment.\n\nPatient question: What are the treatments for this condition and how serious is it"}, {'role': 'assistant', 'content': "As a dermatology assistant, I've analyzed an image and detected Pityriasis Rosea with 100.00% confidence.\n\nPatient details:\n- Age: 22\n\nTailor your response to the patient's age 